In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_kidney_disease"
cohort = "GSE69438"

# Input paths
in_trait_dir = "../../input/GEO/Chronic_kidney_disease"
in_cohort_dir = "../../input/GEO/Chronic_kidney_disease/GSE69438"

# Output paths
out_data_file = "../../output/preprocess/Chronic_kidney_disease/GSE69438.csv"
out_gene_data_file = "../../output/preprocess/Chronic_kidney_disease/gene_data/GSE69438.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_kidney_disease/clinical_data/GSE69438.csv"
json_path = "../../output/preprocess/Chronic_kidney_disease/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
from typing import Optional, Dict, Any, Callable
import json

# Analyzing gene expression data availability
# The background information mentions kidney tissue transcriptome data
# This suggests gene expression data is likely available
is_gene_available = True

# Variable availability and data type conversion
# From the sample characteristics dictionary, there's only information about the tissue source
# There's no direct information about the trait (CKD), age, or gender

# For trait: Based on the background information, this dataset is about Chronic Kidney Disease,
# but the sample characteristics don't explicitly list which samples have CKD vs controls
# Since the Series_overall_design mentions "Chronic Kidney Disease" and various nephropathies,
# we can infer this is a study of CKD and its subtypes, but cannot determine trait status for individual samples
trait_row = None

# For age: No information available
age_row = None

# For gender: No information available
gender_row = None

# Since we can't extract trait information, we don't need conversion functions
# but we'll define them as required
def convert_trait(value):
    if value is None:
        return None
    value = str(value).lower()
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Since we don't have actual trait data, this function is a placeholder
    return None

def convert_age(value):
    if value is None:
        return None
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    if value is None:
        return None
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    value = str(value).lower()
    if "female" in value or "f" == value:
        return 0
    elif "male" in value or "m" == value:
        return 1
    return None

# Save metadata using the validate_and_save_cohort_info function
# Since trait_row is None, is_trait_available is False
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Since trait_row is None, we skip the clinical feature extraction step


### Step 3: Gene Data Extraction

In [ ]:
# Check if the dataset contains gene expression data based on previous assessment
if not is_gene_available:
    print("This dataset does not contain gene expression data (only miRNA data).")
    print("Skipping gene expression data extraction.")
else:
    # Get the matrix file directly rather than using geo_get_relevant_filepaths
    files = os.listdir(in_cohort_dir)
    if len(files) > 0:
        matrix_file = os.path.join(in_cohort_dir, files[0])
        print(f"Matrix file found: {matrix_file}")
        
        try:
            # Extract gene data
            gene_data = get_genetic_data(matrix_file)
            print(f"Gene data shape: {gene_data.shape}")
            
            # Print the first 20 gene/probe identifiers
            print("First 20 gene/probe identifiers:")
            print(gene_data.index[:20])
        except Exception as e:
            print(f"Error extracting gene data: {e}")
    else:
        print("No files found in the input directory.")


### Step 4: Gene Identifier Review

In [ ]:
# Analyzing the gene identifiers shown in the output
# These appear to be Entrez Gene IDs (numeric identifiers), not gene symbols
# For example, '1' is the Entrez ID for A1BG gene, '10' is for NAT2, etc.
# These need to be mapped to human gene symbols for better interpretability

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Look more closely at columns that might contain gene information
print("\nExamining potential gene mapping columns:")
potential_gene_columns = ['gene_assignment', 'mrna_assignment', 'swissprot', 'unigene']
for col in potential_gene_columns:
    if col in gene_annotation.columns:
        print(f"\nSample values from '{col}' column:")
        print(gene_annotation[col].head(3).tolist())


### Step 6: Gene Identifier Mapping

In [ ]:
# Determine which columns to use for mapping
# From the gene annotation preview, we can see:
# - 'ID' column contains the numeric identifiers (same as in gene expression data)
# - 'ORF' column contains the gene symbols we want to map to

# 1. Create the mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='ORF')
print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("First few rows of mapping dataframe:")
print(mapping_df.head())

# 2. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10].tolist())

# 3. Save the gene expression data to a CSV file for future use
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# Since we already determined this dataset doesn't have trait information (trait_row is None),
# we need to skip the clinical data extraction and data linking steps.

# 1. Normalize gene symbols in the index
print("\nNormalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print("First 10 gene identifiers after normalization:")
print(normalized_gene_data.index[:10].tolist())

# Save the normalized gene data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# For the final validation, we set is_trait_available to False since trait_row is None
is_gene_available = len(normalized_gene_data) > 0
is_trait_available = False  # Confirmed in step 2

# Create a dummy dataframe with required structure for validation
dummy_df = pd.DataFrame({trait: [None]})

# Set is_biased to True since the dataset is unusable due to missing trait information
is_biased = True

# Create a note about the dataset
note = "This dataset contains gene expression data for Chronic Kidney Disease, but lacks trait information distinguishing between cases and controls. The dataset cannot be used for the current trait study without clinical annotations."

# 5. Conduct final quality validation and save relevant information
print("\nConducting final quality validation...")
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=dummy_df,
    note=note
)

print(f"Dataset usability: {is_usable}")
print("No linked data saved as dataset is not usable for the current trait study due to missing trait information.")